<a href="https://colab.research.google.com/github/ARADHYA299/OpensourceLLMprojects/blob/main/StudentNotesHelperAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -q faiss-cpu langchain sentence-transformers transformers
!pip install -q pymupdf
!pip install -U -q langchain-community
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00


In [3]:
import fitz
def extract_from_pdf(pdf_path):
  doc = fitz.open(pdf_path)
  text = ""

  for page in doc:
    text += page.get_text()
  return text

In [5]:
from langchain.text_splitter import CharacterTextSplitter

def split_text(text):
  splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,
    chunk_overlap = 100
  )

  return splitter.split_text(text)

In [22]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS

def embed_texts(chunks):
  embedder = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2")
  faiss_index = FAISS.from_texts(chunks , embedding = embedder)
  return faiss_index

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

def load_local_llm():
    model_name = "distilgpt2"  # VERY lightweight, runs everywhere

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Required for distilgpt2 to avoid warning about pad_token
    tokenizer.pad_token = tokenizer.eos_token

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    return HuggingFacePipeline(pipeline=pipe)


In [12]:
from langchain.chains import RetrievalQA

def create_qa_chain(faiss_index , llm):
  return RetrievalQA.from_chain_type(
    llm = llm ,
    retriever = faiss_index.as_retriever()
  )

In [27]:
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

text = extract_from_pdf(pdf_path)
chunks = split_text(text)
faiss_index = embed_texts(chunks)
llm = load_local_llm()
qa_chain = create_qa_chain(faiss_index, llm)

# Example
print(qa_chain.run("What is this document about?"))


Saving Financial markets  Capital markets.pdf to Financial markets  Capital markets (6).pdf


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-26-37d6da3c2f16>:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=pipe)
<ipython-input-27-46ae027d4e90>:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa_chain.run("What is this document about?"))


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

of, and to regulate, the securities market and for matters connected therewith or incidental 
thereto”. In addition to the intent behind the establishment of the Board, Section 11 of the Act 
lays down the functions of the Board that clearly illustrate “it shall be the duty of the board to 
protect the interests of investors in securities and to promote the development of, and to regulate 
the securities market, by such measures as it thinks fit”.

• 
It evaluates the actual worth of securities and provides the intrinsic value of the 
company to avoid deceitful investments  
57  
  
• 
Safeguarding general public interest by ensuring equitable allotment, easy 
transfer, disclosure of proper information, etc.,  
• 
Assures the existence of good faith or an absence of fraud concerning the issue 
of securities.  
• 
Providing a

In [30]:
import gradio as gr

# Keep variables outside the function for persistence
llm = load_local_llm()
vectorstore = None
qa_chain = None

def process_pdf(file):
    global vectorstore, qa_chain
    if file is None:
        return "Please upload a valid PDF file."
    try:
        # Convert to string path if needed
        file_path = file.name if hasattr(file, 'name') else str(file)
        text = extract_from_pdf(file_path)
        chunks = split_text(text)
        vectorstore = embed_texts(chunks)
        qa_chain = create_qa_chain(vectorstore, llm)
        return "✅ PDF uploaded and processed successfully. You can now ask questions!"
    except Exception as e:
        return f"❌ Error processing PDF: {str(e)}"
def ask_question(question):
    if qa_chain is None:
        return "Please upload and process a PDF first!"
    return qa_chain.run(question)

with gr.Blocks() as demo:
    gr.Markdown("# 📄 RAG Chatbot with Local LLM")
    gr.Markdown("Upload a PDF and ask questions about it. Powered by distilGPT2 and FAISS.")

    with gr.Row():
        with gr.Column():
            pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
            process_button = gr.Button("Process PDF")
            process_output = gr.Textbox(label="Status")

        with gr.Column():
            question_input = gr.Textbox(label="Ask a question")
            ask_button = gr.Button("Get Answer")
            answer_output = gr.Textbox(label="Answer")

    process_button.click(fn=process_pdf, inputs=pdf_input, outputs=process_output)
    ask_button.click(fn=ask_question, inputs=question_input, outputs=answer_output)

demo.launch()

Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aaa4dfd454a20d67cb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
